# Contour Tests

In [1]:
# Type imports
from typing import Any, Dict, List, Tuple

# Standard Libraries
from pathlib import Path
from collections import Counter, defaultdict
import math
from math import sqrt, pi, sin, cos, tan, radians
from statistics import mean
from itertools import zip_longest

# Shared Packages
import pandas as pd
import numpy as np
import xlwings as xw

import pygraphviz as pgv
import networkx as nx

In [2]:
from types_and_classes import *

## functions for creating contours

In [3]:
def circle_points(radius: float, offset_x: float = 0, offset_y: float = 0,
                  num_points: int = 16, precision=3)->list[tuple[float, float]]:
    deg_step = radians(360/num_points)
    degree_points = np.arange(stop=radians(360), step=deg_step)
    x_coord = np.array([round(radius*sin(d), precision) for d in degree_points])
    y_coord = np.array([round(radius*cos(d), precision) for d in degree_points])

    x_coord = x_coord + offset_x
    y_coord = y_coord + offset_y
    coords = [(x,y) for x,y in zip(x_coord,y_coord)]
    return coords

In [4]:
def box_points(width:float, height: float = None, offset_x: float = 0,
               offset_y: float = 0) -> list[tuple[float, float]]:
    x1_unit = width / 2
    if not height:
        y1_unit = x1_unit
    else:
        y1_unit = height / 2
    coords = [
        ( x1_unit + offset_x,  y1_unit + offset_y),
        ( x1_unit + offset_x, -y1_unit + offset_y),
        (-x1_unit + offset_x, -y1_unit + offset_y),
        (-x1_unit + offset_x,  y1_unit + offset_y)
        ]
    return coords

In [5]:
def make_contour_slices(roi_num: ROI_Num, slices: List[SliceIndex],
                        structure_2d: StructureSlice):
    data_list = []
    for slice_idx in slices:
        data_item = {
            'ROI Num': roi_num,
            'Slice Index': SliceIndex(slice_idx),
            'Structure Slice': structure_2d
            }
        data_list.append(data_item)
    slice_data = pd.DataFrame(data_list)
    slice_data.set_index(['ROI Num', 'Slice Index'], inplace=True)
    return slice_data

In [6]:
def make_slice_list(number: int, start: float = 0.0, spacing: float = 0.1):
    slices = [round(SliceIndex(num*spacing + start), PRECISION)
              for num in range(number)]
    return slices


In [7]:
def get_exterior(poly: shapely.MultiPolygon)-> shapely.MultiPolygon:
    polygons = [shapely.Polygon(bdr) for bdr in poly.boundary.geoms]
    return shapely.MultiPolygon(polygons)

In [8]:
def get_hull(poly: shapely.MultiPolygon)-> shapely.MultiPolygon:
    hulls = [shapely.convex_hull(p) for p in poly.geoms]
    return shapely.MultiPolygon(polygons)

In [46]:
def make_slice_table(slice_data: pd.DataFrame)->pd.DataFrame:
    slice_table = slice_data.unstack('ROI Num')
    #slice_table.columns = slice_table.columns.droplevel()
    slice_table = slice_table.swaplevel(axis='columns')

    return slice_table

In [ ]:
def relate_structs(slice_table: pd.DataFrame, struct_num: ROI_Num,
                   other_struct_num: ROI_Num):

    def compare(mpoly1, mpoly2):
        relation_str = shapely.relate(mpoly1, mpoly2)
        # Convert relationship string in the form '212FF1FF2' into a boolean string.
        relation_bool = relation_str.replace('F','0').replace('2','1')
        return relation_bool

    def relate(slice_structures: pd.Series, struct_num: ROI_Num,
            other_struct_num: ROI_Num):
        other_structure = slice_structures[(other_struct_num, 'Structure Slice')]
        if other_structure is None:
            return None
        primary_structure = slice_structures[(struct_num, 'Structure Slice')]
        if primary_structure is None:
            return None
        primary_relation = compare(primary_structure, other_structure)
        external_structure = slice_structures[(struct_num, 'Exterior')]
        external_relation = compare(external_structure, other_structure)
        convex_hull_structure = slice_structures[(struct_num, 'Hull')]
        convex_hull_relation = compare(convex_hull_structure, other_structure)

        full_relation = ''.join([convex_hull_relation,
                                external_relation,
                                primary_relation])
        binary_relation = int(full_relation, base=2)
        return bin(binary_relation)

    slice_structures = slice_table.loc[:, [struct_num, other_struct_num]]
    slice_structures.dropna(inplace=True)
    relation_binary = slice_structures.agg(relate, axis='columns',
                                           struct_num=struct_num,
                                           other_struct_num=other_struct_num)
    return relation_binary

In [10]:
def slice_spacing(contour):
    # Index is the slice position of all slices in the image set
    # Columns are structure IDs
    # Values are the distance (INF) to the next contour
    inf = contour.dropna().index.min()
    sup = contour.dropna().index.max()
    contour_range = (contour.index <= sup) & (contour.index >= inf)
    slices = contour.loc[contour_range].dropna().index.to_series()
    gaps = slices.shift(-1) - slices
    return gaps

### Make test structures

In [11]:
# 6 cm x 6 cm box
box6 = shapely.MultiPolygon([
    shapely.Polygon(box_points(6
                               ))])
slices = make_slice_list(5)
slices_1 = make_contour_slices(1, slices, box6)

# 4 cm x 4 cm box
box4 = shapely.MultiPolygon([
    shapely.Polygon(box_points(4))
    ])
slices = make_slice_list(5)
#slices = make_slice_list(5, start=0.2, spacing=0.2)
slices_2 = make_contour_slices(2, slices, box4)

# 6 cm x 6 cm box offset right by 3 cm
offset_box6 = shapely.MultiPolygon([
    shapely.Polygon(box_points(6, offset_x=3))
    ])
slices = make_slice_list(5)
#slices = make_slice_list(5, start=0.2, spacing=0.2)
slices_3 = make_contour_slices(3, slices, offset_box6)

# 6 cm x 6 cm box with 4cm x 4 cm hole
box6 = shapely.Polygon(box_points(6))
box4 = shapely.Polygon(box_points(4))
hollow_box = shapely.MultiPolygon([shapely.difference(box6, box4)])
slices = make_slice_list(5, start=0.2)
slices_4 = make_contour_slices(4, slices, hollow_box)

# 2 2x2 boxes
box2a = shapely.Polygon(box_points(4, offset_x=-3))
box2b = shapely.Polygon(box_points(4, offset_x=3))
box_pair = shapely.MultiPolygon([box2a, box2b])
slices = make_slice_list(5)
#slices = make_slice_list(5, start=0.2, spacing=0.2)
slices_5 = make_contour_slices(5, slices, box_pair)


In [12]:
# combine the slice data
slice_data = pd.concat([slices_1, slices_2, slices_3, slices_4, slices_5])

exterior_contours = slice_data['Structure Slice'].apply(get_exterior)
exterior_contours.name = 'Exterior'
hull_contours = slice_data['Structure Slice'].apply(shapely.convex_hull)
hull_contours.name = 'Hull'

slice_data = pd.concat([slice_data, exterior_contours, hull_contours],
                       axis='columns')
# convert slice data into a table of slices and structures
slice_table = make_slice_table(slice_data)

In [123]:
relate_structs(slice_table, 1, 2)

Slice Index
0.0    0b111001001111001001111001001
0.1    0b111001001111001001111001001
0.2    0b111001001111001001111001001
0.3    0b111001001111001001111001001
0.4    0b111001001111001001111001001
dtype: object

In [116]:
a = shapely.relate(slice_table.at[0, (1, 'Structure Slice')], slice_table.at[0, (2, 'Structure Slice')])
a

'212FF1FF2'

In [ ]:
b = a.replace('F','0').replace('2','1')
c = int(b, base=2)
bin(c)

In [ ]:
mask = 0b101010110
mask

342

In [112]:
value = 0b101000000

In [113]:
(c & mask) == value

True

In [106]:
int(b, base=2)

457

In [107]:
bin(int(b, base=2))

'0b111001001'

In [60]:
slice_table = slice_data.index.to_frame()
slice_table = slice_table['Slice Index'].unstack('ROI Num')
contour_slices = slice_table.apply(slice_spacing)

## Create slice index


In [11]:
def build_slice_table(contour_sets)->pd.DataFrame:
    def form_table(slice_index):
        slice_index.reset_index(inplace=True)
        slice_index.sort_values('Slice', inplace=True)
        slice_index.set_index(['Slice','StructureID'], inplace=True)
        slice_table = slice_index.unstack()
        slice_table.columns = slice_table.columns.droplevel()
        return slice_table

    slice_index = build_contour_index(contour_sets)
    slice_table = form_table(slice_index)
    contour_slices = slice_table.apply(slice_spacing)
    return contour_slices

38

In [ ]:
def build_contour_index(contour_sets: Dict[int, ContourSet])->pd.DataFrame:
    '''Build an index of structures in a contour set.

    The table columns contain the structure names, the ROI number, and the
    slice positions where the contours for that structure are located.  There
    is one row for each slice and structure on that slice.  Multiple contours
    for a single structure on a given slice, have only one row in teh contour
    index

    Args:
        contour_sets (Dict[int, RS_DICOM_Utilities.ContourSet]): A dictionary
            of structure data.

    Returns:
        pd.DataFrame: An index of structures in a contour set indication which
            slices contains contours for each structure.
    '''
    slice_ref = {}
    name_ref = {}
    for structure in contour_sets.values():
        slice_ref[structure.roi_num] = list(structure.contours.keys())
        name_ref[structure.roi_num] = structure.structure_id
    slice_seq = pd.Series(slice_ref).explode()
    slice_seq.name = 'Slice'
    name_lookup = pd.Series(name_ref)
    name_lookup.name = 'StructureID'
    slice_lookup = pd.DataFrame(name_lookup).join(slice_seq, how='outer')
    return slice_lookup

In [ ]:
structure_2d: StructureSlice = shapely.MultiPolygon([hollow_box])

ContourData: Table
	Index: AutoInteger
	Columns:
		ROI_Num,
		SliceIndex,
		Area,
		Contour
Generated by: Read Contour Data

StructureData: Series:
	Index: ROI_Num, SliceIndex
	Values: StructureSlice
Generated by: Build StructureSet


In [ ]:
box1_def = {
        'struct_id': 'GTV',
        'roi': 38,
        'structure_type': 'GTV',
        'structure_code': 'GTVp',
        'structure_code_meaning': 'Primary Gross Tumor Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 0, 0),
        'volume': 8.03,
        'length': 2.6,
        'sup_slice': -0.4,
        'inf_slice': -3,
        'center_of_mass': (-5.36,  9.71, -1.63)
        }

structure_def = Structure(**box1_def)


In [ ]:
structure_def = [
    {
        'struct_id': 'GTV',
        'roi': 38,
        'structure_type': 'GTV',
        'structure_code': 'GTVp',
        'structure_code_meaning': 'Primary Gross Tumor Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 0, 0),
        'volume': 8.03,
        'length': 2.6,
        'sup_slice': -0.4,
        'inf_slice': -3,
        'center_of_mass': (-5.36,  9.71, -1.63)
        },{
        'struct_id': 'CTV',
        'roi': 24,
        'structure_type': 'GTV',
        'structure_code': 'ITV',
        'structure_code_meaning': 'Internal Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 255, 0),
        'volume': 34.45,
        'length': 3.6,
        'sup_slice': 0,
        'inf_slice': -3.6,
        'center_of_mass': (-5.34,  9.79, -1.61)
        },{
        'struct_id': 'PTV',
        'roi': 30,
        'structure_type': 'PTV',
        'structure_code': 'PTVp',
        'structure_code_meaning': 'Primary Planning Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 255),
        'volume': 74.649,
        'length': 4.6,
        'sup_slice': 0.6,
        'inf_slice': -4.0,
        'center_of_mass': (-5.36,  9.78, -1.59)
        },{
        'struct_id': 'eval PTV',
        'roi': 41,
        'structure_type': 'PTV',
        'structure_code': 'PTVp',
        'structure_code_meaning': 'Primary Planning Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 255),
        'volume': 74.649,
        'length': 4.6,
        'sup_slice': 0.6,
        'inf_slice': -4.0,
        'center_of_mass': (-5.36,  9.78, -1.59)
        },{
        'struct_id': 'BODY',
        'roi': 1,
        'structure_type': 'EXTERNAL',
        'structure_code': 'BODY',
        'structure_code_meaning': 'Body',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 0),
        'volume': 28951.626,
        'length': 33.8,
        'sup_slice': 10.6,
        'inf_slice': -23.2,
        'center_of_mass': (-0.95,  9.73, -6.76)
        },{
        'struct_id': 'Lung L',
        'roi': 26,
        'structure_type': 'ORGAN',
        'structure_code': '7310',
        'structure_code_meaning': 'Left lung',
        'structure_code_scheme': 'FMA',
        'color': (224, 255, 255),
        'volume': 1776,
        'length': 24.2,
        'sup_slice': 5.8,
        'inf_slice': -18.4,
        'center_of_mass': (7.08, 10.61, -6.28)
        },{
        'struct_id': 'Lung R',
        'roi': 27,
        'structure_type': 'ORGAN',
        'structure_code': '7309',
        'structure_code_meaning': 'Right lung',
        'structure_code_scheme': 'FMA',
        'color': (255, 218, 185),
        'volume': 2556.676,
        'length': 23.8,
        'sup_slice': 6.6,
        'inf_slice': -17.2,
        'center_of_mass': (-8.09,  8.77, -5.57)
        },{
        'struct_id': 'Lung B',
        'roi': 25,
        'structure_type': 'ORGAN',
        'structure_code': '68877',
        'structure_code_meaning': 'Pair of lungs',
        'structure_code_scheme': 'FMA',
        'color': (218, 165, 32),
        'volume': 4332.676,
        'length': 25,
        'sup_slice': 6.6,
        'inf_slice': -18.4,
        'center_of_mass': (-1.87,  9.52, -5.86)
        },{
        'struct_id': 'Skin',
        'roi': 2,
        'structure_type': 'ORGAN',
        'structure_code': '7163',
        'structure_code_meaning': 'Skin',
        'structure_code_scheme': 'FMA',
        'color': (240, 255, 240),
        'volume': 1726.808,
        'length': 33.8,
        'sup_slice': 10.6,
        'inf_slice': -23.2,
        'center_of_mass': (-0.95,  9.73, -6.76),
        'show': False
        }
    ]

structure_list = []

for node_dict in structure_def:
    roi = node_dict['roi']
    struct_id = node_dict['struct_id']
    structure_ref = {'roi': roi, 'struct_id': struct_id,
                     'Structure': Structure(**node_dict)}
    structure_list.append(structure_ref)

structure_table = pd.DataFrame(structure_list)
structure_table.set_index('roi', inplace=True)

In [ ]:
edge_def = [
    {'structures': (1, 2),   'relationship': 'CONFINES',      'is_logical': False, 'show': True},
    {'structures': (1, 25),  'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (1, 27),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 26),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 30),  'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (1, 41),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 24),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (1, 38),  'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (2, 25),  'relationship': 'DISJOINT',      'is_logical': False, 'show': True},
    {'structures': (2, 27),  'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (2, 26),  'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (2, 30),  'relationship': 'SURROUNDS',     'is_logical': False, 'show': False},
    {'structures': (2, 41),  'relationship': 'SURROUNDS',     'is_logical': True , 'show': True},
    {'structures': (2, 24),  'relationship': 'SURROUNDS',     'is_logical': True , 'show': False},
    {'structures': (2, 38),  'relationship': 'SURROUNDS',     'is_logical': True , 'show': False},
    {'structures': (25, 27), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    {'structures': (25, 26), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    {'structures': (25, 30), 'relationship': 'OVERLAPS',      'is_logical': True , 'show': False},
    {'structures': (25, 41), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (25, 24), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (25, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (27, 26), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 30), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 41), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 24), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (27, 38), 'relationship': 'DISJOINT',      'is_logical': False, 'show': False},
    {'structures': (26, 30), 'relationship': 'OVERLAPS',      'is_logical': False, 'show': True},
    {'structures': (26, 41), 'relationship': 'INCORPORATES',  'is_logical': True , 'show': True},
    {'structures': (26, 24), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (26, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': False},
    {'structures': (30, 41), 'relationship': 'INCORPORATES',  'is_logical': False, 'show': True},
    {'structures': (30, 24), 'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (30, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (41, 24), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    {'structures': (41, 38), 'relationship': 'CONTAINS',      'is_logical': True , 'show': True},
    {'structures': (24, 38), 'relationship': 'CONTAINS',      'is_logical': False, 'show': True},
    ]


relationship_list = []

for edge_dict in edge_def:
    relationship = Relationship(**edge_dict)
    relationship_type = edge_dict['relationship']
    edge = list(edge_dict['structures']) + [relationship_type, relationship]
    relationship_list.append(edge)
relationship_table = pd.DataFrame(relationship_list)
relationship_table.columns = ['ROI_1', 'ROI_2', 'Relationship Type',
                              'Relationship']
relationship_table.set_index(['ROI_1', 'ROI_2'], inplace=True)